In [4]:
import numpy as np

In [5]:
def wrapper(lambda_function):
    
    def wrapped_function(args: list):
        return lambda_function(*args)
    
    return wrapped_function

In [6]:
def max_error(system: list, args: list) -> float:

    return max([abs(lam(*args)) for lam in system])

In [84]:
def Jacobi(system: list, values=None, eps=1e-7) -> list:
    '''
        The simple iteration method (Jacobi method). 
        Example:
            Input:
                system = [
                    lambda x, y, z: x ** 2 + np.sin(y) + z,
                    lambda x, y, z: y ** 2 + np.cos(z) + x, 
                    lambda x, y, z: z ** 2 + y / x + 2
                ]
                start_values = [1., 1., 1.] (if None - automatic answer search)
            Output:
                [x1, x2, x3]
    '''

    n = len(system)
    answer = []
    min_error = np.inf   
    start_bias = np.zeros((n, ))
    wsystem = [wrapper(lam) for lam in system]
    epoches = 100 ** n 
    if values is None:
        values = np.ones((n, ))
    else:
        if not isinstance(values, np.ndarray):
            values = np.array(values)
        epoches = 1

    for i in range(epoches):
        k = -1 ** (i % 2)
        values *= k
        start_bias *= k
        values = values + start_bias
        start_bias += 0.001
        error_last = np.inf / 2
        iter = 0
        while error_last > eps and iter < 1000:
            iter += 1
            values = [values[i] - wsystem[i](values) for i in range(n)] # Вычисление i-ого приближения 
            error_cur = max_error(system, values) 
            if (error_cur > error_last) or (error_cur > np.inf) or None in values:
                values = np.ones((n, ))
                break
            elif error_cur < min_error:
                answer = values
                min_error = error_cur
            error_last = error_cur                
                
        if error_cur < eps:
            return values, min_error
        
    return answer, min_error

In [86]:
print(Jacobi([lambda x: x ** 3 - 1], [10]))

([1.0], 0.0)


In [87]:
system = [
    lambda x, y, z: x ** 2 + np.sin(y) + z,
    lambda x, y, z: y ** 2 + np.cos(z) + x, 
    lambda x, y, z: z ** 2 + y / x + 2
]
print(Jacobi(system))

KeyboardInterrupt: 